In [1]:
import numpy as np
from TMDP import TMDP
from river_swim import River
import pprint

In [3]:
gamma = 0.9
river = River(gamma)
tau = 0.9
xi = np.ones(river.nS)*1/river.nS
tmdp = TMDP(river, xi, tau, gamma)
print(tmdp.P_mat)
print(tmdp.P_mat_tau)

[1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1.]


AttributeError: 'TMDP' object has no attribute 'P_mat'